In [1]:
from __future__ import division
import numpy as np
import scipy.io as scipy
import matplotlib.pyplot as plt
!pip install mne
import mne
from numpy import *
from numpy.fft import *
import scipy.signal as signal
from matplotlib.pyplot import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import sklearn

In [2]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], scipy.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, scipy.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [31]:
p2_all = loadmat('P2_AllLifts.mat')
print( "All Lifts loaded")



#these are the series we are looking at for subject 2. Everything except series 1 and 6
trials_included=[2, 3, 4, 5, 7, 8, 9] 

#corresponding row indices in P.AllLifts where each series starts. Index is off by one bc of how python indexes things
trialStartIdx = [28, 62, 96, 130, 192, 226, 260]

#data structure to hold all EEG trials 
eeg_data_raw = []
emg_data_raw = []
series_counter = 0
trial_counter = 1
i = 0
for s in trials_included:
    # load all the series specified in trials_included
    series_filename = 'WS_P2_S{series}.mat'.format(series = s)
    p_series = loadmat(series_filename)
    print("Series data loaded")
    series_counter = series_counter + 1
    print("series_counter: ", series_counter)

    #identifying the starting row index in p.ALLLifts for the first trial of the current series
    tStart = trialStartIdx[i]
    i += 1
    trial_counter = 1
    #range for the for loop is 34 values bc there are 34 trials in each series
    for trial_num in range(tStart, tStart + 34):
        
        #retrieving the point in time when both fingers touched object
        #index values are one cloumn and one row before target value
        tBothDigitTouch = np.array(p2_all['P']['AllLifts'])[trial_num, 14]
        
        tBothReleased = np.array(p2_all['P']['AllLifts'])[trial_num, 22]
        
        #converting time points to row indices to be accessed in windowed struct
        #500 samples every second, so timepoint should be divided by 0.0002
        idxStart = int(tBothDigitTouch // 0.002)
        idxEnd = int(tBothReleased // 0.002)

        eeg_trial = np.array(p_series['ws']['win'][0]['eeg'])[idxStart:idxEnd].T 
        eeg_data_raw.append(eeg_trial)
        emg_trial = np.array(p_series['ws']['win'][0]['emg'])[idxStart:idxEnd].T 
        emg_data_raw.append(emg_trial)
        #print("trial counter: ", trial_counter)
        trial_counter = trial_counter +1
                                                            

All Lifts loaded
Series data loaded
series_counter:  1
trial counter:  1
trial counter:  2
trial counter:  3
trial counter:  4
trial counter:  5
trial counter:  6
trial counter:  7
trial counter:  8
trial counter:  9
trial counter:  10
trial counter:  11
trial counter:  12
trial counter:  13
trial counter:  14
trial counter:  15
trial counter:  16
trial counter:  17
trial counter:  18
trial counter:  19
trial counter:  20
trial counter:  21
trial counter:  22
trial counter:  23
trial counter:  24
trial counter:  25
trial counter:  26
trial counter:  27
trial counter:  28
trial counter:  29
trial counter:  30
trial counter:  31
trial counter:  32
trial counter:  33
trial counter:  34
Series data loaded
series_counter:  2
trial counter:  1
trial counter:  2
trial counter:  3
trial counter:  4
trial counter:  5
trial counter:  6
trial counter:  7
trial counter:  8
trial counter:  9
trial counter:  10
trial counter:  11
trial counter:  12
trial counter:  13
trial counter:  14
trial counter

In [11]:
len(eeg_data_raw)

238

In [32]:
len(emg_data_raw)

238

In [28]:
series = [1, 2, 3, 4, 5, 6, 7]
new_dict = {}
counter = 0
for i in range (1, len(series)+1):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(eeg_data_raw[s])
    new_dict[i] = new_list
print(new_dict.keys())
print(new_dict.values())


dict_keys([1, 2, 3, 4, 5, 6, 7])
dict_values([[array([[-1069., -1130., -1119., ...,  -932., -1016., -1054.],
       [ -110.,  -166.,  -177., ...,   218.,   169.,   162.],
       [    6.,   -61.,   -41., ...,    47.,   -62.,   -41.],
       ...,
       [  263.,   144.,   238., ...,   209.,    96.,    58.],
       [  191.,    90.,   216., ...,    32.,   -76.,   -81.],
       [  383.,   302.,   536., ...,   -30.,    55.,   200.]],
      dtype=float32), array([[-1087., -1145., -1119., ...,  -907.,  -859.,  -867.],
       [  -78.,  -146.,  -127., ...,   229.,   269.,   289.],
       [ -121.,  -189.,  -170., ...,   116.,   147.,   151.],
       ...,
       [  212.,   113.,   110., ...,   273.,   230.,   140.],
       [  196.,    87.,    80., ...,   132.,    61.,   -79.],
       [  443.,   352.,   270., ...,   456.,   108.,  -260.]],
      dtype=float32), array([[-1342., -1309., -1258., ...,  -584.,  -603.,  -652.],
       [ -358.,  -314.,  -244., ...,   520.,   512.,   458.],
       [ -269.,

In [34]:
series = [1, 2, 3, 4, 5, 6, 7]
new_dict = {}
counter = 0
for i in range (1, len(series)+1):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(emg_data_raw[s])
    new_dict[i] = new_list
print(new_dict.keys())
print(new_dict.values())


dict_keys([1, 2, 3, 4, 5, 6, 7])
dict_values([[array([[ 0.00774259,  0.01049259,  0.01049259, ..., -0.00095741,
        -0.00045741,  0.00393259],
       [-0.0008464 , -0.0006464 , -0.0022464 , ...,  0.0031536 ,
         0.0051056 ,  0.0090736 ],
       [-0.00548585, -0.00258585, -0.00628585, ...,  0.00486415,
         0.00486415,  0.00181415],
       [ 0.00684849,  0.00761849,  0.00116849, ...,  0.00486849,
         0.00276849,  0.00136849],
       [-0.0012883 , -0.0007883 , -0.0006883 , ...,  0.0007117 ,
        -0.0065883 , -0.0065883 ]]), array([[-0.00625741, -0.00435741, -0.00065741, ...,  0.00074259,
         0.00314259,  0.00054259],
       [-0.0042464 ,  0.0022536 , -0.0008464 , ..., -0.0043464 ,
         0.0011536 ,  0.0012536 ],
       [-0.00608585, -0.00658585, -0.00958585, ...,  0.00516615,
         0.00745415,  0.00791415],
       [ 0.00056849,  0.00395849,  0.00486849, ...,  0.00086849,
         0.00306849,  0.00116849],
       [-0.0007883 , -0.0007883 , -0.0007883 , ...,